# Construcción del modelo documental

En este notebook se transforman los datasets previamente limpiados en una
estructura documental adaptada a MongoDB.

Se aplica un modelo desnormalizado embebido:

Documento principal → Local
Subdocumentos → Licencias, Terrazas y Actividad económica

El objetivo es generar un único documento por local que contenga toda la
información relacionada, evitando joins y optimizando consultas.


In [ ]:
import pandas as pd
import json
import pprint
from pathlib import Path

## Carga de datasets normalizados

Se cargan las versiones *clean* generadas en la fase anterior.
En este punto los datos ya tienen tipos consistentes y pueden ser
transformados sin ambigüedades.


In [ ]:
df_terrazas = pd.read_csv("../../data/cleaned/terrazas_clean.csv")
df_locales = pd.read_csv("../../data/cleaned/locales_clean.csv")
df_licencias = pd.read_csv("../../data/cleaned/licencias_clean.csv")
df_actividad = pd.read_csv("../../data/cleaned/actividad_clean.csv")

print("Datasets cargados")


## Conversión de valores nulos

Pandas utiliza el valor especial NaN para representar datos faltantes.
Sin embargo, este valor no es válido en JSON ni en MongoDB.

MongoDB utiliza `null`, equivalente a `None` en Python.

Por ello se reemplazan todos los NaN por None antes de generar los documentos.
Esto evita errores de inserción y mantiene la semántica de "dato desconocido".


In [ ]:
def df_to_python_nulls(df):
    return df.astype(object).where(pd.notnull(df), None)


df_terrazas = df_to_python_nulls(df_terrazas)
df_locales = df_to_python_nulls(df_locales)
df_licencias = df_to_python_nulls(df_licencias)
df_actividad = df_to_python_nulls(df_actividad)

print("NaN convertidos a None correctamente")


In [ ]:
print(df_locales.columns)
print(df_terrazas.columns)
print(df_licencias.columns)
print(df_actividad.columns)


## Identificador de relación entre datasets

Durante la exploración inicial se identificó que `id_local` aparece en los
cuatro ficheros y actúa como nexo común.

Por tanto, se utilizará como clave principal para reconstruir las relaciones
entre entidades en el modelo documental.


In [ ]:
CLAVE = "id_local"

## Selección de atributos relevantes

No todos los atributos originales son necesarios para el modelo documental.
Se seleccionan aquellos que:

- Identifican el local
- Permiten realizar las consultas solicitadas
- Describen las características principales

El resto de información podría almacenarse en futuras versiones del modelo
si fuese necesario.


In [ ]:
local_cols = [
    "id_local",
    "desc_distrito_local",
    "desc_barrio_local",
    "desc_tipo_acceso_local",
    "desc_situacion_local",
    "clase_vial_edificio",
    "desc_vial_edificio",
    "rotulo",
    "cod_postal",
    "coordenada_x_local",
    "coordenada_y_local",
    "hora_apertura1",
    "hora_cierre1"
]

terraza_cols = [
    "id_local",
    "id_terraza",
    "desc_periodo_terraza",
    "desc_ubicacion_terraza",
    "Superficie_ES",
    "mesas_es",
    "sillas_es",
    "Fecha_confir_ult_decreto_resol"
]

lic_cols = [
    "id_local",
    "ref_licencia",
    "desc_tipo_licencia",
    "desc_tipo_situacion_licencia",
    "Fecha_Dec_Lic"
]

act_cols = [
    "id_local",
    "desc_seccion",
    "desc_division",
    "desc_epigrafe"
]


## Agrupación de información relacionada

En un modelo relacional los datos estarían separados en tablas.
Aquí se reconstruyen las relaciones agrupando por `id_local`.

Cada agrupación generará un array de subdocumentos:
- Un local → varias licencias
- Un local → varias terrazas
- Un local → varias actividades económicas


In [ ]:
licencias_group = {
    clave: grupo.drop(columns=[CLAVE], errors="ignore").to_dict("records")
    for clave, grupo in df_licencias[lic_cols].groupby(CLAVE)
}

print("Agrupación de licencias por id_local se ha realizado correctamente")

In [ ]:
terrazas_group = {
    clave: grupo.drop(columns=[CLAVE], errors="ignore").to_dict("records")
    for clave, grupo in df_terrazas[terraza_cols].groupby(CLAVE)
}

print("Agrupación de terrazas por id_local se ha realizado correctamente")

In [ ]:
actividad_group = {
    clave: grupo.drop(columns=[CLAVE], errors="ignore").to_dict("records")
    for clave, grupo in df_actividad[act_cols].groupby(CLAVE)
}

print("Agrupación de actividad económica por id_local se ha realizado correctamente")

## Construcción del documento principal

Se genera un documento por cada local y se insertan dentro de él los arrays
de subdocumentos obtenidos previamente.

El resultado es un modelo desnormalizado embebido:

```
Local
├── Licencias[]
├── Terrazas[]
└── Actividad económica[]
```

Esto elimina la necesidad de joins y optimiza las consultas analíticas.


In [ ]:
# Preparar solo las columnas necesarias de locales
df_locales_final = df_locales[local_cols]

documentos = []
# Convertir a records primero es más rápido que iterrows
for doc in df_locales_final.to_dict("records"):
    local_id = doc[CLAVE]
    
    # Asignar los grupos (usando .get para evitar errores si no existe la clave)
    doc["licencias"] = licencias_group.get(local_id, [])
    doc["terrazas"] = terrazas_group.get(local_id, [])
    doc["actividad_economica"] = actividad_group.get(local_id, [])
    
    documentos.append(doc)

print("Documentos generados:", len(documentos))

## Verificación del documento generado

Se inspecciona manualmente un documento para comprobar que la estructura es
correcta y que los subdocumentos han sido incrustados adecuadamente.


In [ ]:
pprint.pprint(documentos[1])

## Exportación a formato JSON Lines

Se utiliza el formato JSONL (un documento por línea) en lugar de un único JSON
gigante porque:

- MongoDB importa este formato directamente
- Reduce el uso de memoria
- Permite manejar grandes volúmenes de datos
- Evita errores de tamaño máximo de documento


In [ ]:

#definimos la ruta de salida
OUTPUT_DIR = Path("../../data/output")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

output_file = OUTPUT_DIR / "locales.jsonl"

# Guardar los documentos en formato JSONL
with open(output_file, "w", encoding="utf-8") as f:
    for doc in documentos:
        f.write(json.dumps(doc, ensure_ascii=False) + "\n")

print("Archivo generado en:", output_file.resolve())